In [1]:
## Kflod-5
import os
import torch
import torchvision
import albumentations as A
from albumentations.pytorch import ToTensor, ToTensorV2
import torchvision.transforms as T

import time
import csv
import numpy as np
import torch.optim as optim
import torch.nn as nn
from PIL import Image
from args import build_opt
from loaders import build_loader, build_loader_new
from tqdm.notebook import tqdm
# from torch.utils.tensorboard import SummaryWriter
opt = build_opt()

from utils import create_logger
import timm
## logger
logger = create_logger('../logging')
##
from sklearn.model_selection import KFold
import csv
import pandas as pd
from itertools import islice
import wandb

...........................................................................
...param: traindir: ../train/labeled_data/
...param: train_metadir: ../
...param: train_metafile: train_label.csv
...param: testdir: ../test
...param: test_metadir: None
...param: test_metafile: None
...param: epochs: 10
...param: num_workers: 4
...param: batch_size: 64
...param: lr: 0.1
...param: weight_decay: 1e-05
...param: num_classes: 2
...param: step_size: 5
...param: gamma: 0.9
...param: ckpt_name: None
...param: submission_name: submission.csv
...param: model_name: None
...param: kfold: 5
...param: device: cuda
...param: resultdir: ./result
...........................................................................


In [2]:
## 重置opt
opt.epochs = 5
opt.lr = 1e-3
opt.model_name = 'efficientnet_b6'
opt.batch_size = 16
opt.kfold = 5
## random seed
torch.manual_seed(42)
np.random.seed(42)


## DEA

In [3]:
model_list = timm.list_models('*swin*')
# model = timm.create_model('efficientnet_b3', pretrained=True)
model_list

['eca_swinnext26ts_256',
 'swin_base_patch4_window7_224',
 'swin_base_patch4_window7_224_in22k',
 'swin_base_patch4_window12_384',
 'swin_base_patch4_window12_384_in22k',
 'swin_large_patch4_window7_224',
 'swin_large_patch4_window7_224_in22k',
 'swin_large_patch4_window12_384',
 'swin_large_patch4_window12_384_in22k',
 'swin_small_patch4_window7_224',
 'swin_tiny_patch4_window7_224',
 'swinnet26t_256',
 'swinnet50ts_256']

In [21]:
train_df = pd.read_csv('../train_new.csv', index_col=0)
train_df.head()
train_df['label'].value_counts()


0    25767
6     1719
2      533
4      269
7      255
5      247
3      223
1      151
Name: label, dtype: int64

In [7]:
# with open('../train_label.csv', 'r') as f:
#     rows = f.readlines()
#     f.close()

In [ ]:
# train_imgs = []
# train_labels = []

# for row in tqdm(rows, total= len(rows)):
#     row = row.rstrip('\n')
#     print(row)
#     train_img = row.split(',')[0]
#     # print(train_img)
#     train_lbl = row.split(',')[1]
#     # print(train_lbl)
#     train_imgs.append(train_img)
#     train_labels.append(train_lbl)
    

In [17]:
# train_df_multilbl = pd.DataFrame({'train_img': train_imgs, 
#                                  'label': train_labels
#                                  })

In [5]:
train_df['label'].value_counts() # 8 class

0    25767
6     1719
2      533
4      269
7      255
5      247
3      223
1      151
Name: label, dtype: int64

## TRAIN

In [22]:
wandb.login()

wandb: Currently logged in as: qddse. Use `wandb login --relogin` to force relogin


True

In [23]:
run = wandb.init(
    project='HUAWEI_lane_rendering', 
    entity='qddse',
    name='Effi-b6-baseline_KFold4_with_aug_multilbl',
    reinit=True,
    config=opt,
    group='EffiNetb6',
    job_type='train'
)

## update config
# api = wandb.Api()
# run = api.run('qddse/HUAWEI_lane_rendering/1us1v6o4')
# run.config['model_name'] = opt.model_name
# run.update()

In [24]:
# GAIN Train
# ckpt_middle = torch.load('../ckpts/8_22/efficientnet_b6_39_bestmodel.pth')
# model.load_state_dict(ckpt_middle['model'])

import gc
gc.collect()


398

## Trainer

In [25]:
## Trainer
class Trainer:
    def __init__(self, dataloaders, optimizer, model, loss_fn, scheduler=None, device='cuda:0', amp=False, config=None):
        self.train_loader, self.val_loader = dataloaders
        self.train_loss = loss_fn
        self.scheduler = scheduler
        self.optimizer = optimizer
        self.device = device
        self.amp = amp
        self.config = config
        self.model = model
        
    def train_one_epoch(self):
        self.model.cuda()
        self.model.train()
        train_pbar = tqdm(enumerate(self.train_loader), total=len(self.train_loader))
        train_preds, train_labels = [], []
        running_loss = 0
        for bnum, data_cache in train_pbar:
            img = data_cache[0].cuda()
            label = data_cache[1].long().cuda()
            ## compute loss
            pred = self.model(img)
            loss = self.train_loss(pred, label)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            running_loss += loss.item()
        return running_loss / (len(self.train_loader))
    
    @torch.no_grad()
    def valid_one_epoch(self):
        self.model.eval()
        valid_pbar = tqdm(enumerate(self.val_loader), total=len(self.val_loader))
        valid_preds, valid_targets = [], []
        valid_acc = 0.
        for idx, cache in valid_pbar:
            img = cache[0].cuda()
            target = cache[1].cuda()
            pred = self.model(img)
            valid_acc += (pred.argmax(1) == target).sum().item()
            
        return valid_acc / len(self.val_loader.dataset)
            
    def train_one_epoch_with_mixup(self):
        '''
        Train one epoch using mixup
        '''
        if self.amp:
            scaler = GradScaler()
        self.model.train()
        train_pbar = tqdm(enumerate(self.train_loader), total=len(self.train_loader))
        train_preds, train_labels = [], []
        running_loss = 0
        for bnum, data_cache in train_pbar:
            img = self._convert_if_not_tensor(data_cache[0], dtype=torch.float32)
            target = self._convert_if_not_tensor(data_cache[1], dtype=torch.float32)

            bs = img.shape[0]
            ## Support Amp with Mixup
            if self.amp:
                # Mixup
                if torch.randn(1)[0] < 0.5:
                    mix_img, tar_a, tar_b, lam = mixup_augmentation(img, target, alpha=0.5)
                    with autocast(enabled=True):
                        output = self.model(mix_img).squeeze()
                        ## mixup loss
                        loss_a = self.train_loss_fn(output, tar_a)
                        loss_b = self.train_loss_fn(output, tar_b)
                        loss = loss_a * lam + (1-lam) * loss_b
                        
                        loss = loss / self.config['N_ACCUM']
                    scaler.scale(loss).backward()
                    if (bnum + 1) % self.config['N_ACCUM'] == 0:
                        scaler.step(self.optimizer)
                        scaler.update()
                        optimizer.zero_grad()
                        
                        if self.scheduler:
                            self. scheduler.step()
                    running_loss += (loss.item() * bs)
                    
                else:
                    with autocast(enabled=True):
                        output = self.model(img).squeeze()
                        loss = self.train_loss_fn(output, target)
                        loss = loss / self.config['N_ACCUM']
                    scaler.scale(loss).backward()
                    
                    if (bnum + 1) % self.config['N_ACCUM'] == 0:
                        scaler.step(self.optimizer)
                        scaler.update()
                        optimizer.zero_grad()
                        
                        if self.scheduler:
                            self.scheduler.step()
                    running_loss += (loss.item() * bs)
                    
            else:  # no amp
                ## mixup allowed
                if torch.randn(1)[0] < 0.5:
                    mix_img, tar_a, tar_b, lam = mixup_augmentation(img, target, alpha=0.5)
                    output = self.model(mix_img).squeeze()
                    
                    ## loss
                    loss_a = self.train_loss_fn(output, tar_a)
                    loss_b = self.train_loss_fn(output, tar_b)
                    loss = loss_a * lam + (1-lam) * loss_b
                    loss = loss / self.config['N_ACCUM']
                    loss.backward()
                    
                    if (bnum + 1)% self.config['N_ACCUM'] == 0:
                        self.optimizer.step()
                        optimizer.zero_grad()
                        
                        if self.scheduler:
                            self.scheduler.step()
                    running_loss += (loss.item() * bs)
                else:
                    output = self.model(img).squeeze()
                    loss = self.train_loss_fn(output, target)
                    loss = loss / self.config['N_ACCUM']
                    loss.backward()

                    if (bnum + 1) % self.config['N_ACCUM'] == 0:
                        self.optimizer.step()
                        optimizer.zero_grad()

                        if self.scheduler:
                            self.scheduler.step()
                    running_loss += (loss.item() * bs)
            train_pbar.set_description(desc=f'loss: {loss.item():.4f}')
            running_loss /= len(self.train_loader)
            
            # Rescale the targets and output before chugging in a matrix
            output = output.sigmoid().detach() * 100.0
            target = target.detach() * 100.0
            train_preds += [output.cpu().numpy()]
            train_labels += [target.cpu().numpy()]
        
        all_train_preds = np.concatenate(train_preds)
        all_train_labels = np.concatenate(train_labels)
        
        # Tidy
        del output, target, train_preds, train_labels, loss, img, meta, all_train_preds, all_train_labels
        gc.collect()
        torch.cuda.empty_cache()
        
        return running_loss
    
    def fit(self, fold: int=None, epochs:int = 10, output_dir:str='./ckpts/826/04/', custom_name:str='model_best.pth', gain_train=None, is_wandb=True):
        """
        Low-effort alternative for doing the complete training and validation process
        args:
            gain_train: str ,ckpts的地址
        """
        ## scheduler
        self.optimizer
        best_loss = int(1e7)
        best_acc = 0.0
        if gain_train:
            ckpt = torch.load(gain_train)
            model_ckpt = ckpt['model']
            self.model.load_state_dict(model_ckpt)
        for epoch in range(epochs):
            print(f"{'*'*20} Epoch: {epoch} / {epochs} {'*'*20}")
            
            train_running_loss = self.train_one_epoch()
            print(f"Training loss: {train_running_loss:.4f}")
            valid_acc = self.valid_one_epoch()
            print(f"Validation Acc : {valid_acc:.4f}")
            
            if train_running_loss < best_loss:
                best_loss = train_running_loss
                self.save_model(output_dir, custom_name, loss_final=best_loss)
                self.save_model(output_dir + '/fold_'+str(fold) + '/', 'model_' + str(epoch) + '.pth', best_loss)

                print(f"Saved model with train_loss: {best_loss:.4f}")
                es = 0
            else:
                es += 1
                print("Counter: {} / {}".format(es, self.config['patience']))
                if es > self.config['patience']:
                    print('Early Stop with train_loss: {}, val_acc:{}'.format(train_running_loss, valid_acc))
                    break
            if best_acc < valid_acc:
                best_acc = valid_acc
                self.save_model(output_dir + 'via_acc', custom_name)
        
            ## logger
            if is_wandb:
                run.log({'epoch:': epoch, 'Fold': fold, 'loss_train:': train_running_loss, 'val_acc': valid_acc})
            if self.scheduler:
                self.scheduler.step()
                
    def save_model(self, path, name, verbose=False, loss_final=None):
        """
        Saves the model at the provided destination
        """
        try:
            if not os.path.exists(path):
                os.mkdir(path)
        except:
            print("Errors encountered while making the output directory")
        torch.save({
            'model':self.model.state_dict(), 
            'loss': loss_final,
            },
            os.path.join(path, name)
        )
        if verbose:
            print(f"Model Saved at: {os.path.join(path, name)}")

In [9]:
## 转化为二分类
# def Convert_01(label):
#     '''
#     args:
#         label:int
#     '''
#     if label == 0:
#         return 0
#     else:
#         return 1
    
# ## 将label 转化为0-1
# train_df['label_01'] = train_df['label'].apply(lambda x: Convert_01(x))
# train_df.sample(20)

,image,label,label_01
15505,train_15505.png,0,0
9642,train_09642.png,0,0
23208,train_23208.png,0,0
7917,train_07917.png,0,0
23946,train_23946.png,0,0
13744,train_13744.png,0,0
6531,train_06531.png,0,0
16565,train_16565.png,0,0
23404,train_23404.png,0,0
250,train_00250.png,0,0


In [10]:
# ## test Dataset
# from loaders import Rane_Dataset
# data_set = Rane_Dataset(imagedir='../train/labeled_data/', df=train_df, transform=None)

# loader =build_loader_new(imagedir='../train/labeled_data/', batch_size=32, num_workers=4, metafile=train_df)

# inp = next(iter(loader))
# inp[0].shape, inp[1]

# # inp = data_set[0][0].unsqueeze(0)

# from models import Effnet

# # model = torchvision.models.resnet50(pretrained=False, num_classes=opt.num_classes).to(opt.device)
# model = Effnet(model_name=opt.model_name).cuda()

# out = model(inp[0].cuda())

# target = inp[1]
# target = target.long()

# criterion = nn.CrossEntropyLoss()
# loss = criterion(out, target.cuda())

In [27]:
## train
from sklearn.model_selection import KFold

## modified model    
from models import Effnet

# model = torchvision.models.resnet50(pretrained=False, num_classes=opt.num_classes).to(opt.device)
model = Effnet(model_name=opt.model_name)
optimizer = optim.AdamW(model.parameters(), lr=opt.lr, weight_decay=opt.weight_decay)
loss_fn = nn.CrossEntropyLoss()
# transform = T.Compose([
#                       T.Resize((224, 224)),
#                       #T.HorizontalFlip(p=0.5),
#                       T.RandomHorizontalFlip(0.5),
#                       T.RandomVerticalFlip(0.5),
#                       # A.VerticalFlip(p=0.5),
#                       # T.Random,
#                       # A.RandomContrast(p=0.5),
#                       T.RandomGrayscale(0.5),
#                       # A.RandomBrightness(p=0.5),
#                       T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
#                       T.ToTensor(),
#                 ])
transform = A.Compose([
                      A.Resize(300, 300),
                      A.HorizontalFlip(p=0.5),
                      A.VerticalFlip(p=0.5),
                      A.RandomContrast(p=0.5),
                      A.RandomBrightness(p=0.5),
                      A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                      ToTensorV2(),
                ])
transform_val = A.Compose([
                      A.Resize(300, 300),
                      # A.HorizontalFlip(p=0.5),
                      # A.VerticalFlip(p=0.5),
                      # A.RandomContrast(p=0.5),
                      # A.RandomBrightness(p=0.5),
                      A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                      ToTensorV2(),
                ])
lr_scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0= 20, eta_min=1e-4)

kf = KFold(n_splits=5)

    
# writer = SummaryWriter() 

No pretrained weights exist for this model. Using random initialization.


In [30]:
for fold, split in enumerate(kf.split(train_df)):
    print(f"{'*'*20} Fold: {fold} / {opt.kfold} {'*'*20}")
    if fold == 0 or fold == 1:
        continue
    train_ = train_df.iloc[split[0]]
    val_ = train_df.iloc[split[1]]
    print(train_.shape[0], val_.shape[0])
    
    train_loader = build_loader_new(
        imagedir=opt.traindir,
        batch_size=opt.batch_size,
        num_workers=opt.num_workers,
        # metadir=opt.train_metadir,
        metafile=train_,
        require_label=True,
        transform = transform
    )
    val_loader = build_loader_new(
        imagedir=opt.traindir,
        batch_size=opt.batch_size,
        num_workers=opt.num_workers,
        # metadir=opt.train_metadir,
        metafile=val_,
        require_label=True,
        transform=transform_val
    )
    Train = Trainer((train_loader, val_loader), optimizer, model, loss_fn, scheduler=lr_scheduler)
    Train.fit(fold=fold, epochs=opt.epochs, output_dir='../ckpts/9_04/', gain_train=None)
    del Train
    gc.collect()

******************** Fold: 0 / 5 ********************
******************** Fold: 1 / 5 ********************
******************** Fold: 2 / 5 ********************
23331 5833
******************** Epoch: 0 / 5 ********************


[ WARN:0@810.109] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../train/labeled_data/train_00016.png'): can't open/read file: check file path/integrity
[ WARN:0@810.109] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../train/labeled_data/train_00000.png'): can't open/read file: check file path/integrity
[ WARN:0@810.109] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../train/labeled_data/train_00032.png'): can't open/read file: check file path/integrity
[ WARN:0@810.112] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../train/labeled_data/train_00064.png'): can't open/read file: check file path/integrity
[ WARN:0@810.112] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../train/labeled_data/train_00096.png'): can't open/read file: check file path/integrity
[ WARN:0@810.116] global /io/opencv/modules/imgcodecs/src/loadsav

  0%|          | 0/1459 [00:00<?, ?it/s]

[ WARN:0@810.154] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../train/labeled_data/train_00128.png'): can't open/read file: check file path/integrity


AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/root/miniconda3/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/root/miniconda3/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/root/autodl-tmp/HUWEI/baseline/loaders.py", line 38, in __getitem__
    img = img.astype(np.float32)
AttributeError: 'NoneType' object has no attribute 'astype'


In [ ]:
os.system('shutdown')

## Origin_Train

In [28]:
print('...start training')

for epoch in range(opt.epochs):
    print('------ Epoch: {} ------'.format(epoch))
    
    start_t = time.time()
    min_loss = 100
    epoch_l = 0
    epoch_t = 0
    for batch_idx, batch in tqdm(enumerate(train_loader), total=len(train_loader)):
        optimizer.zero_grad()
        image, label = batch
        image, label = image.to(opt.device), label.to(opt.device)
        output = model(image)

        l = loss(output, label)
        l.backward()
        optimizer.step()

        batch_l = l.item()
        epoch_l += batch_l
        batch_t = time.time() - start_t
        epoch_t += batch_t
        start_t = time.time()
    lr_scheduler.step()
    epoch_t = epoch_t / len(train_loader)
    epoch_l = epoch_l / len(train_loader)
    print('...epoch: {:3d}/{:3d}, loss: {:.4f}, average time: {:.2f}.'.format(
        epoch + 1, opt.epochs, epoch_l, epoch_t))
    ## WandB logging
    run.log({'epoch:': epoch, 'loss_train:': epoch_l})
    run.log({'epoch:': epoch, 'time_train:': epoch_t})
    ## save model
    if epoch_l < min_loss:
        save_name = opt.model_name + '_' + str(epoch+20) + '_bestmodel.pth'
        torch.save({
            'model': model.state_dict(),
            'lr': opt.lr,
            'loss': epoch_l},
            os.path.join('../ckpts/8_25', save_name)
        )
        min_lossoss = epoch_l
        ## Early-stop
        es = 0
    else:
        es += 1
        print('Counter {}/{}'.format(es, 4))  # 设置patience为4
        if es > 3:
            print('Early_stop with best model‘s running_loss:{}'.format(epoch_l))
            break
        

...start training
------ Epoch: 0 ------


NameError: name 'train_loader' is not defined

## SAVE CHECKPOINT

In [ ]:
# torch.save({
#     'model': model.state_dict(),
#     'lr': '0.1*10 + 1e-4*10',
#     'epoch': '10+10',},
#     '../ckpts/res50_epoch10+10'
#     )

In [ ]:
import os
os.system("shutdown")

## INFERENCE

In [4]:
## test_loader
test_loader = build_loader(
    imagedir=opt.testdir,
    batch_size=opt.batch_size,
    num_workers=opt.num_workers,
    metadir=opt.test_metadir,
    metafile=opt.test_metafile,
    require_label=False
)

## 加载模型
model = torchvision.models.resnet50(pretrained=False, num_classes=opt.num_classes).to(opt.device)
ckpt = torch.load('../ckpts/baseline_724.pth')['model']
model.load_state_dict(ckpt)

print('...start predicting')

model.eval()
to_prob = nn.Softmax(dim=1)
with torch.no_grad():
    imagenames, probs = list(), list()
    for batch_idx, batch in tqdm(enumerate(test_loader), total=len(test_loader)):
        image, imagename = batch
        image = image.to(opt.device)
        pred = model(image)
        prob = to_prob(pred)
        prob = list(prob.data.cpu().numpy())
        imagenames += imagename
        probs += prob

with open(os.path.join(opt.resultdir, 'submission_old.csv'), 'w', encoding='utf8') as fp:
    writer = csv.writer(fp)
    writer.writerow(['imagename', 'defect_prob'])
    for info in zip(imagenames, probs):
        imagename, prob = info
        writer.writerow([imagename, str(prob[1])])

...start predicting


  0%|          | 0/63 [00:00<?, ?it/s]